In [2]:
# Install necessary packages
!pip install langchain faiss-cpu sentence-transformers transformers datasets pypdf rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.2/358.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install -U langchain-community
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [4]:
from google.colab import files
uploaded = files.upload()

Saving 13. Atlas of Diabetes Mellitus (3rd Edition).pdf to 13. Atlas of Diabetes Mellitus (3rd Edition).pdf


In [5]:
#Step 1: Load and Split PDF

pdf_path = "/content/13. Atlas of Diabetes Mellitus (3rd Edition).pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

print(f"✅ Total Chunks: {len(chunks)}")


✅ Total Chunks: 738


In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer

# ✅ LangChain-compatible wrapper for building FAISS index
embedding_model = HuggingFaceEmbeddings(model_name='pritamdeka/S-PubMedBert-MS-MARCO')

# ✅ Raw SentenceTransformer model for encoding queries in hybrid_retrieve
raw_dense_model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')

# Texts from chunks
texts = [doc.page_content for doc in chunks]

# ✅ Create dense FAISS index using LangChain wrapper
vector_store = FAISS.from_texts(texts, embedding_model)

# ✅ Create sparse BM25 index
tokenized_texts = [text.split(" ") for text in texts]
bm25 = BM25Okapi(tokenized_texts)


<ipython-input-6-66f15360052e>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name='pritamdeka/S-PubMedBert-MS-MARCO')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dat

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/461k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256, temperature=0.7)
llm = HuggingFacePipeline(pipeline=pipe)


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-7-ef02839a93e3>:9: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [8]:
def hybrid_retrieve(query, top_k=4):
    # Sparse
    query_tokens = query.lower().split()
    bm25_scores = bm25.get_scores(query_tokens)
    sparse_indices = np.argsort(bm25_scores)[::-1][:top_k]

    # Dense
    dense_query = raw_dense_model.encode([query], convert_to_numpy=True)[0]  # ✅ Correct
    _, dense_indices = vector_store.index.search(np.array([dense_query]), top_k)

    # Merge
    hybrid_indices = list(set(sparse_indices).union(set(dense_indices[0])))
    hybrid_docs = [chunks[i] for i in hybrid_indices]
    return hybrid_docs


In [9]:
import csv
import os

def log_evaluation_to_csv(filepath, question, answer, sources, faithfulness_score, relevance_score):
    file_exists = os.path.isfile(filepath)
    with open(filepath, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['Question', 'Answer', 'Sources', 'Faithfulness', 'Relevance'])
        combined_sources = "\n---\n".join([doc.page_content[:500] for doc in sources])
        writer.writerow([question, answer, combined_sources, faithfulness_score, relevance_score])

EVAL_TEMPLATE = """
You are an evaluator. Rate the following answer using the source documents provided.

Question: {question}

Answer: {answer}

Sources:
{sources}

Give a score from 1 to 3 for:
1. FAITHFULNESS (Is it accurate?)
2. RELEVANCE (Does it answer the question?)

Format:
FAITHFULNESS: <score>, RELEVANCE: <score>
"""

def get_auto_scores(llm, question, answer, sources):
    source_text = "\n---\n".join([doc.page_content[:500] for doc in sources])
    prompt = EVAL_TEMPLATE.format(question=question, answer=answer, sources=source_text)
    response = llm(prompt)

    import re
    try:
        match = re.search(r"FAITHFULNESS: (\d).*RELEVANCE: (\d)", response[0]['generated_text'], re.DOTALL)
        faithfulness = int(match.group(1))
        relevance = int(match.group(2))
    except Exception as e:
        print("⚠️ Parse error:", e)
        return None, None

    return faithfulness, relevance


In [11]:
query = "What are the main types of diabetes and how are they different?"
retrieved_docs = hybrid_retrieve(query)
context = "\n\n".join([doc.page_content[:500] for doc in retrieved_docs])

prompt = f"Answer the following question based on the context.\n\nContext:\n{context}\n\nQuestion: {query}"
answer = llm(prompt)

print("\n📝 Question:", query)
print("\n✅ Answer:\n", answer)
print("\n📚 Source Snippets:\n")
for i, doc in enumerate(retrieved_docs):
    print(f"Source {i+1}:\n{doc.page_content[:300]}\n---")


# Auto Evaluation
faithfulness_score, relevance_score = get_auto_scores(llm, query, answer, retrieved_docs)
print(f"\n🤖 Auto Evaluation — Faithfulness: {faithfulness_score}, Relevance: {relevance_score}")

# Log
log_evaluation_to_csv('qa_hybrid_evals.csv', query, answer, retrieved_docs, faithfulness_score, relevance_score)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Token indices sequence length is longer than the specified maximum sequence length for this model (2195 > 2048). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



📝 Question: What are the main types of diabetes and how are they different?

✅ Answer:
 Answer the following question based on the context.

Context:
over the past two decades. The nomenclature has changed from insulin-dependent diabetes to type 1 diabetes and from non-
insulin diabetes mellitus to type 2 diabetes. All forms of diabetes are characterized according to their known etiologies,
immunologic, genetic or otherwise. This opens up the concept of ‘the diabetic syndrome’. HNF, hepatic nuclear factor; MODY,
maturity-onset diabetes of the young
Scobie Chapter 01  23/8/06  11:40  Page 6

seen in adult-onset type 1 and 2 diabetes mellitus (DM), it
is most commonly seen in children and young adults with
type 1 DM. The development of this abnormality is linked
to the duration of diabetes. When present, other diabetic
complications are likely to coexist
Scobie Chapter 07  31/7/06  10:22  Page 106

(IGT).
CLASSIFICATION OF DIABETES
The diagnostic label ‘diabetes mellitus’ refers not to 

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


⚠️ Parse error: string indices must be integers, not 'str'

🤖 Auto Evaluation — Faithfulness: None, Relevance: None
